In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
date = "2024-12-30"

In [0]:
from pyspark.sql.functions import current_timestamp, lit, concat, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("languageId", IntegerType(), True),
    StructField("languageRoleId", IntegerType(), True)
])

In [0]:
movie_language_df = spark.read \
                .schema(schema) \
                .option("multiLine", "True") \
                .json(f'{bronze_folder_path}/{date}/movie_language')

In [0]:
movie_language_final_df = movie_language_df \
                            .withColumnRenamed("movieId", "movie_id") \
                            .withColumnRenamed("languageId", "language_id") \
                            .withColumn("ingestion_date", current_timestamp()) \
                            .withColumn("env", lit("python")) \
                            .drop(col('languageRoleId')) \
                            .withColumn('file_date', lit(date))

In [0]:
movie_language_final_df.write.mode("overwrite").format("parquet").saveAsTable('movie_silver.movies_languages')